In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import datetime
start_date = datetime.datetime.now() - datetime.timedelta(days=3*365)
stock = yf.download("LT.NS",start = start_date,end = datetime.datetime.now())
stock.head()

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()
[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-02-01,1345.750000,1462.0,1337.199951,1448.849976,1411.337158,11317043
2021-02-02,1474.000000,1593.0,1474.000000,1520.199951,1480.839600,14349665
2021-02-03,1530.000000,1558.0,1505.000000,1528.550049,1488.973633,6197782
2021-02-04,1525.000000,1550.0,1510.000000,1529.800049,1490.191162,3957579
2021-02-05,1534.400024,1547.0,1511.099976,1516.500000,1477.235596,3270494


In [ ]:
stock = stock[stock['Volume'] != 0]
stock['daily_change'] = stock['Close'].diff()
#https://blog.quantinsti.com/rsi-indicator/#:~:text=For%20example%2C%20for%20(14%2D,manner%2C%20the%20table%20is%20updated.&text=While%20the%20formula%20for%20RSI,Average%20gain%20or%20loss%20values.
def rsi_cal(length):
  stock['gain'] = stock['daily_change'].apply(lambda x: x if x > 0 else 0)
  stock['loss'] = stock['daily_change'].apply(lambda x: -x if x < 0 else 0)
  stock['rsi'] = 100 - (100/(1+(stock['gain'].rolling(window=length).mean()/stock['loss'].rolling(window=length).mean())))
  stock.drop(columns='gain',inplace = True)
  stock.drop(columns='loss',inplace = True)

rsi_cal(14)
stock


<ipython-input-3-1892dcb5e083>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock['daily_change'] = stock['Close'].diff()
<ipython-input-3-1892dcb5e083>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock['gain'] = stock['daily_change'].apply(lambda x: x if x > 0 else 0)
<ipython-input-3-1892dcb5e083>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

,Open,High,Low,Close,Adj Close,Volume,daily_change,rsi
Date,,,,,,,,
2021-02-01,1345.750000,1462.000000,1337.199951,1448.849976,1411.337158,11317043,NaN,NaN
2021-02-02,1474.000000,1593.000000,1474.000000,1520.199951,1480.839600,14349665,71.349976,NaN
2021-02-03,1530.000000,1558.000000,1505.000000,1528.550049,1488.973633,6197782,8.350098,NaN
2021-02-04,1525.000000,1550.000000,1510.000000,1529.800049,1490.191162,3957579,1.250000,NaN
2021-02-05,1534.400024,1547.000000,1511.099976,1516.500000,1477.235596,3270494,-13.300049,NaN
...,...,...,...,...,...,...,...,...
2024-01-19,3603.000000,3655.850098,3603.000000,3646.000000,3646.000000,1913923,50.000000,62.160824
2024-01-23,3637.600098,3661.350098,3536.199951,3551.000000,3551.000000,1742142,-95.000000,61.215980
2024-01-24,3570.000000,3617.949951,3521.699951,3589.199951,3589.199951,3711521,38.199951,63.801581


In [ ]:
#https://machinelearningmastery.com/how-to-use-correlation-to-understand-the-relationship-between-variables/
from scipy.stats import pearsonr
from scipy.stats import spearmanr
def correl_(x,y):
  new_data = stock.dropna(subset=[y])
  correlation_matrix = stock[[x, y]].corr()
  print(f'correlation matrix {correlation_matrix}')
  corr= pearsonr(new_data[x], new_data[y])
  print(f'pearsonr {corr}')
  corr= spearmanr(new_data[x], new_data[y])
  print(f'spearnamr {corr}')

correl_('Close','rsi')


correlation matrix           Close       rsi
Close  1.000000  0.318129
rsi    0.318129  1.000000
pearsonr PearsonRResult(statistic=0.3181294403472198, pvalue=1.5455297783885863e-18)
spearnamr SignificanceResult(statistic=0.34080633239410346, pvalue=3.3434967706342126e-21)


In [ ]:
def moving_average(length):
  stock['ma'+str(length)] = stock['Close'].rolling(window=length).mean()

moving_average(20)
moving_average(50)
stock.tail()

<ipython-input-5-5cbf90ad65e4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock['ma'+str(length)] = stock['Close'].rolling(window=length).mean()
<ipython-input-5-5cbf90ad65e4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock['ma'+str(length)] = stock['Close'].rolling(window=length).mean()


,Open,High,Low,Close,Adj Close,Volume,daily_change,rsi,ma20,ma50
Date,,,,,,,,,,
2024-01-19,3603.000000,3655.850098,3603.000000,3646.000000,3646.000000,1913923,50.000000,62.160824,3527.312512,3345.830005
2024-01-23,3637.600098,3661.350098,3536.199951,3551.000000,3551.000000,1742142,-95.000000,61.215980,3530.965015,3356.921006
2024-01-24,3570.000000,3617.949951,3521.699951,3589.199951,3589.199951,3711521,38.199951,63.801581,3535.922510,3368.208003
2024-01-25,3599.899902,3618.250000,3564.800049,3593.449951,3593.449951,2206540,4.250000,62.829682,3538.395007,3379.412002
2024-01-29,3604.000000,3733.850098,3600.100098,3708.000000,3708.000000,2072982,114.550049,66.140525,3547.892505,3392.602002


In [ ]:
def calculate_bollinger_bands(k, window_size):
    stock['st_deviation'] = stock['Close'].rolling(window=window_size).std()
    stock['bandwidth'] = k * stock['st_deviation']
    stock['bollinger_UB'] = stock['ma20'] + stock['bandwidth']
    stock['bollinger_LB'] = stock['ma20'] - stock['bandwidth']
    stock.drop(columns='st_deviation',inplace = True)
    stock.drop(columns='bandwidth',inplace = True)

calculate_bollinger_bands(2,20)
stock.tail()

<ipython-input-6-af5ee19291b1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock['st_deviation'] = stock['Close'].rolling(window=window_size).std()
<ipython-input-6-af5ee19291b1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock['bandwidth'] = k * stock['st_deviation']
<ipython-input-6-af5ee19291b1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

,Open,High,Low,Close,Adj Close,Volume,daily_change,rsi,ma20,ma50,bollinger_UB,bollinger_LB
Date,,,,,,,,,,,,
2024-01-19,3603.000000,3655.850098,3603.000000,3646.000000,3646.000000,1913923,50.000000,62.160824,3527.312512,3345.830005,3631.545583,3423.079441
2024-01-23,3637.600098,3661.350098,3536.199951,3551.000000,3551.000000,1742142,-95.000000,61.215980,3530.965015,3356.921006,3633.011600,3428.918429
2024-01-24,3570.000000,3617.949951,3521.699951,3589.199951,3589.199951,3711521,38.199951,63.801581,3535.922510,3368.208003,3639.225732,3432.619288
2024-01-25,3599.899902,3618.250000,3564.800049,3593.449951,3593.449951,2206540,4.250000,62.829682,3538.395007,3379.412002,3644.831829,3431.958186
2024-01-29,3604.000000,3733.850098,3600.100098,3708.000000,3708.000000,2072982,114.550049,66.140525,3547.892505,3392.602002,3677.961047,3417.823962


In [ ]:
def calculate_macd():
  stock['ShortEMA'] = stock['Close'].ewm(span=12, adjust=False).mean()
  stock['LongEMA'] = stock['Close'].ewm(span=26, adjust=False).mean()
  stock['MACD'] = stock['ShortEMA'] - stock['LongEMA']
  signal_window = 9
  stock['Signal_Line'] = stock['MACD'].ewm(span=signal_window, adjust=False).mean()
  stock['macd_diff'] = stock['MACD'] - stock['Signal_Line']
  stock.drop(columns='ShortEMA',inplace = True)
  stock.drop(columns='LongEMA',inplace = True)
  stock.drop(columns='MACD',inplace = True)
  stock.drop(columns='Signal_Line',inplace = True)

calculate_macd()
stock.tail()

<ipython-input-7-0de8755e3861>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock['ShortEMA'] = stock['Close'].ewm(span=12, adjust=False).mean()
<ipython-input-7-0de8755e3861>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock['LongEMA'] = stock['Close'].ewm(span=26, adjust=False).mean()
<ipython-input-7-0de8755e3861>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

,Open,High,Low,Close,Adj Close,Volume,daily_change,rsi,ma20,ma50,bollinger_UB,bollinger_LB,macd_diff
Date,,,,,,,,,,,,,
2024-01-19,3603.000000,3655.850098,3603.000000,3646.000000,3646.000000,1913923,50.000000,62.160824,3527.312512,3345.830005,3631.545583,3423.079441,-0.728246
2024-01-23,3637.600098,3661.350098,3536.199951,3551.000000,3551.000000,1742142,-95.000000,61.215980,3530.965015,3356.921006,3633.011600,3428.918429,-4.981423
2024-01-24,3570.000000,3617.949951,3521.699951,3589.199951,3589.199951,3711521,38.199951,63.801581,3535.922510,3368.208003,3639.225732,3432.619288,-5.561161
2024-01-25,3599.899902,3618.250000,3564.800049,3593.449951,3593.449951,2206540,4.250000,62.829682,3538.395007,3379.412002,3644.831829,3431.958186,-5.962001
2024-01-29,3604.000000,3733.850098,3600.100098,3708.000000,3708.000000,2072982,114.550049,66.140525,3547.892505,3392.602002,3677.961047,3417.823962,0.822982


In [ ]:
#-1 correspond to sell
# 1 correspond to buy
# 0 correspond to neutral
def bollinger_signal():
  stock['bollinger_buy_sell'] = np.where(stock['Close'] > stock['bollinger_UB'], -1, np.where(stock['Close'] < stock['bollinger_LB'], 1, 0))

def rsi_signal():
  stock['rsi_buy_sell'] = np.where(stock['rsi'] > 65, -1, np.where(stock['rsi'] < 35, 1, 0))

def macd_signal():
  stock['macd_buy_sell'] = np.where(stock['macd_diff'] >= 0, -1, np.where(stock['macd_diff'] < 0, 1, 0))

def ma_signal():
  stock['ma_buy_sell'] = 0
  stock.loc[stock['ma20'] >= stock['ma50'], 'ma_buy_sell'] = 1
  stock.loc[stock['ma20'] < stock['ma50'], 'ma_buy_sell'] = -1

def calculate_real_market_indicator_2BS():
    stock['Real_Market_indicator_2BS'] = 0
    stock.loc[stock['daily_change'] >= 0 , 'Real_Market_indicator_2BS'] = 1
    stock.loc[stock['daily_change'] < 0, 'Real_Market_indicator_2BS'] = -1
    stock['Real_Market_indicator_2BS'] = stock['Real_Market_indicator_2BS'].shift(-1)

bollinger_signal()
rsi_signal()
macd_signal()
ma_signal()
calculate_real_market_indicator_2BS()
stock



<ipython-input-8-4ae8f0c8b5af>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock['bollinger_buy_sell'] = np.where(stock['Close'] > stock['bollinger_UB'], -1, np.where(stock['Close'] < stock['bollinger_LB'], 1, 0))
<ipython-input-8-4ae8f0c8b5af>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock['rsi_buy_sell'] = np.where(stock['rsi'] > 65, -1, np.where(stock['rsi'] < 35, 1, 0))
<ipython-input-8-4ae8f0c8b5af>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

,Open,High,Low,Close,Adj Close,Volume,daily_change,rsi,ma20,ma50,bollinger_UB,bollinger_LB,macd_diff,bollinger_buy_sell,rsi_buy_sell,macd_buy_sell,ma_buy_sell,Real_Market_indicator_2BS
Date,,,,,,,,,,,,,,,,,,
2021-02-01,1345.750000,1462.000000,1337.199951,1448.849976,1411.337158,11317043,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,0,-1,0,1.0
2021-02-02,1474.000000,1593.000000,1474.000000,1520.199951,1480.839600,14349665,71.349976,NaN,NaN,NaN,NaN,NaN,4.553389,0,0,-1,0,1.0
2021-02-03,1530.000000,1558.000000,1505.000000,1528.550049,1488.973633,6197782,8.350098,NaN,NaN,NaN,NaN,NaN,7.691174,0,0,-1,0,1.0
2021-02-04,1525.000000,1550.000000,1510.000000,1529.800049,1490.191162,3957579,1.250000,NaN,NaN,NaN,NaN,NaN,9.306557,0,0,-1,0,-1.0
2021-02-05,1534.400024,1547.000000,1511.099976,1516.500000,1477.235596,3270494,-13.300049,NaN,NaN,NaN,NaN,NaN,8.933286,0,0,-1,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-19,3603.000000,3655.850098,3603.000000,3646.000000,3646.000000,1913923,50.000000,62.160824,3527.312512,3345.830005,3631.545583,3423.079441,-0.728246,-1,0,1,1,-1.0
2024-01-23,3637.600098,3661.350098,3536.199951,3551.000000,3551.000000,1742142,-95.000000,61.215980,3530.965015,3356.921006,3633.011600,3428.918429,-4.981423,0,0,1,1,1.0
2024-01-24,3570.000000,3617.949951,3521.699951,3589.199951,3589.199951,3711521,38.199951,63.801581,3535.922510,3368.208003,3639.225732,3432.619288,-5.561161,0,0,1,1,1.0


In [ ]:
df = stock.copy()
df = df.dropna()
# Select relevant features
c = df[['rsi','ma20','ma50','bollinger_UB','bollinger_LB','macd_diff','bollinger_buy_sell','rsi_buy_sell','macd_buy_sell','ma_buy_sell','Real_Market_indicator_2BS']]

# Calculate the correlation matrix
corr_matrix = c.corr()

corr_matrix

,rsi,ma20,ma50,bollinger_UB,bollinger_LB,macd_diff,bollinger_buy_sell,rsi_buy_sell,macd_buy_sell,ma_buy_sell,Real_Market_indicator_2BS
rsi,1.000000,0.169853,0.147969,0.177749,0.160242,0.760937,-0.361003,-0.898932,-0.627215,0.229222,-0.009267
ma20,0.169853,1.000000,0.993750,0.997628,0.997132,-0.066438,-0.021729,-0.142465,0.093113,0.352764,0.064364
ma50,0.147969,0.993750,1.000000,0.992010,0.990221,-0.042371,-0.015608,-0.123484,0.071126,0.277057,0.070893
bollinger_UB,0.177749,0.997628,0.992010,1.000000,0.989557,-0.049153,-0.014968,-0.152218,0.078884,0.347949,0.066103
bollinger_LB,0.160242,0.997132,0.990221,0.989557,1.000000,-0.085078,-0.029045,-0.130963,0.108248,0.356127,0.062099
macd_diff,0.760937,-0.066438,-0.042371,-0.049153,-0.085078,1.000000,-0.361261,-0.691689,-0.785005,-0.124952,-0.008112
bollinger_buy_sell,-0.361003,-0.021729,-0.015608,-0.014968,-0.029045,-0.361261,1.000000,0.329625,0.297133,-0.132153,0.024944
rsi_buy_sell,-0.898932,-0.142465,-0.123484,-0.152218,-0.130963,-0.691689,0.329625,1.000000,0.584529,-0.218186,0.023742
macd_buy_sell,-0.627215,0.093113,0.071126,0.078884,0.108248,-0.785005,0.297133,0.584529,1.000000,0.114093,-0.001796
ma_buy_sell,0.229222,0.352764,0.277057,0.347949,0.356127,-0.124952,-0.132153,-0.218186,0.114093,1.000000,0.002190


In [ ]:

# Select relevant features (X) and target variable (y)
x = df[['Open','High','Low','Close','Volume','Adj Close','daily_change','rsi','ma20','ma50','bollinger_UB','bollinger_LB','macd_diff']]  # Replace with actual feature columns
# x = df[['bollinger_buy_sell','rsi_buy_sell','macd_buy_sell','ma_buy_sell']] # Replace with actual feature columns
# x = df[['Open','High','Low','Close','Adj Close','Volume','daily_change','rsi','ma20','ma50','bollinger_UB','bollinger_LB','macd_diff','bollinger_buy_sell','rsi_buy_sell','macd_buy_sell','ma_buy_sell']]
y = df['Real_Market_indicator_2BS']

from sklearn.model_selection import train_test_split
# Split the data into training and testing sets (e.g., 80% train, 20% test)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Initialize Logistic Regression model
model = LogisticRegression()

# Fit the model on the training data
model.fit(x_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(x_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_report_output = classification_report(y_test, y_pred)

# Display results
print(f"Accuracy: {accuracy:.2%}")
print("\nClassification Report:\n", classification_report_output)


Accuracy: 57.97%

Classification Report:
               precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00        58
         1.0       0.58      1.00      0.73        80

    accuracy                           0.58       138
   macro avg       0.29      0.50      0.37       138
weighted avg       0.34      0.58      0.43       138



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Import required libraries
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Create a decision tree classifier
clf = DecisionTreeClassifier(random_state=42)

# Train the model using the training data
clf.fit(x_train, y_train)

# Make predictions on the testing data
y_pred = clf.predict(x_test)

# Evaluate the model performance
accuracy = accuracy_score(y_test, y_pred)

# Display results
print(f"Accuracy: {accuracy:.2%}")


Accuracy: 52.17%


In [ ]:
# Import required libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Create a random forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model using the training data
rf.fit(x_train, y_train)

# Make predictions on the testing data
y_pred = rf.predict(x_test)

# Evaluate the model performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 53.62%


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Train the KNN model on the training data
knn_model = KNeighborsClassifier(n_neighbors=5, p=2)
knn_model.fit(x_train, y_train)

# Make predictions on the testing data
y_pred = knn_model.predict(x_test)
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 52.17%
